In [1]:
import os
import pandas as pd
import numpy as np
cwd = os.getcwd()
print(cwd)

/home/rraju/recommender_system/debug_notebook_scripts


In [2]:
df = pd.read_csv('../books_recommender/data/bookclub_events.csv', parse_dates=['event_time', 'receipt_time'])

/home/rraju/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.shape

(23644617, 12)

In [12]:
len(df[(df['learner_id'] == 214103.0) & (df['event_name'] == 'audio_close')]['book_code'].unique())

0

In [4]:
dataframe = df
print("No of records : ", len(dataframe))
dataframe = dataframe.drop_duplicates(['row_id'])
print("No of records : ", len(dataframe))

No of records :  23644617
No of records :  23644461


In [5]:
# eliminate books with null id
dataframe = dataframe[dataframe['book_code'].notnull()]
print("Removed books with null id, No of Records : ", len(dataframe))

# eliminate learners with null id
dataframe = dataframe[dataframe['learner_id'].notnull()]
print("Removed learners with null id, No of Records : ", len(dataframe))

#since no of open and close events are unequal, just filter on close events
events_filter = ((dataframe['event_name'] == 'book_close') |\
                 (dataframe['event_name'] == 'video_close') |\
                 (dataframe['event_name'] == 'audio_close'))
dataframe = dataframe[events_filter]
print("After filtering events for close events, No of Records : ", len(dataframe))

Removed books with null id, No of Records :  13507961
Removed learners with null id, No of Records :  13507961
After filtering events for close events, No of Records :  5566611


In [6]:
dataframe.sort_values(by='event_time', inplace=True)

In [15]:
dataframe.groupby('event_name').agg({'learner_id': (lambda x: len(x.unique())),
                                     'book_code' : (lambda x: len(x.unique()))
                                    }).rename(columns={'learner_id' : 'no_of_learners',
                                                       'book_code' : 'no_of_items'
                                                      }).reset_index()

,event_name,no_of_learners,no_of_items
0,audio_close,5845.0,538
1,book_close,230004.0,7070
2,video_close,152881.0,3882


In [57]:
item_type_stats = dataframe.groupby(['learner_id', 'event_name'])\
                           .agg({'book_code' : (lambda x: len(x.unique()))})\
                           .rename(columns={'book_code' : 'no_of_items'})\
                           .reset_index()

In [58]:
total_item_type_stats = item_type_stats.groupby('learner_id')\
                                       .agg({'no_of_items' : np.sum})\
                                       .rename(columns={'no_of_items' : 'total_no_of_items'})\
                                       .reset_index()

In [59]:
learner_item_type_stats = item_type_stats.merge(total_item_type_stats, on='learner_id')

In [60]:
learner_item_type_stats['percentage'] = learner_item_type_stats['no_of_items'] / learner_item_type_stats['total_no_of_items']

In [63]:
import datetime as DT
demograph = pd.read_csv('../books_recommender/data/bc.demographics.csv', 
                        parse_dates=['learner_birthday'])
now = pd.Timestamp(DT.datetime.now())
demograph['dob'] = pd.to_datetime(demograph['learner_birthday'], format='%m%d%y')
demograph['dob'] = demograph['dob'].where(demograph['dob'] < now, demograph['dob'] -  np.timedelta64(100, 'Y'))
demograph['age'] = (now - demograph['dob']).astype('<m8[Y]')

In [64]:
demograph.head()

,learner_id,learner_birthday,learner_gender,dob,age
0,378403.0,2008-03-06 15:00:00,female,2008-03-06 15:00:00,9.0
1,540035.0,2013-11-24 00:00:00,male,2013-11-24 00:00:00,4.0
2,145904.0,2007-09-02 15:00:00,male,2007-09-02 15:00:00,10.0
3,391266.0,2009-07-12 15:00:00,female,2009-07-12 15:00:00,8.0
4,420946.0,2010-03-10 15:00:00,male,2010-03-10 15:00:00,7.0


In [65]:
learner_age_item_type_stats = learner_item_type_stats.merge(demograph, on='learner_id')

In [66]:
learner_age_item_type_stats

,learner_id,event_name,no_of_items,total_no_of_items,percentage,learner_birthday,learner_gender,dob,age
0,5.0,book_close,2,2,1.000000,2003-06-02 15:00:00,female,2003-06-02 15:00:00,14.0
1,31.0,book_close,1,1,1.000000,2005-10-18 15:00:00,male,2005-10-18 15:00:00,12.0
2,41.0,video_close,1,1,1.000000,2005-02-13 15:00:00,female,2005-02-13 15:00:00,12.0
3,61.0,book_close,22,24,0.916667,1999-10-04 15:00:00,female,1999-10-04 15:00:00,18.0
4,61.0,video_close,2,24,0.083333,1999-10-04 15:00:00,female,1999-10-04 15:00:00,18.0
5,181.0,book_close,1,18,0.055556,2005-08-09 15:00:00,male,2005-08-09 15:00:00,12.0
6,181.0,video_close,17,18,0.944444,2005-08-09 15:00:00,male,2005-08-09 15:00:00,12.0
7,322.0,video_close,1,1,1.000000,2004-03-18 15:00:00,male,2004-03-18 15:00:00,13.0
8,456.0,book_close,3,3,1.000000,2005-05-30 15:00:00,male,2005-05-30 15:00:00,12.0
9,521.0,book_close,18,18,1.000000,2010-03-25 15:00:00,female,2010-03-25 15:00:00,7.0


In [ ]:
learner_age_item_type_stats

In [70]:
learner_age_item_type_stats[learner_age_item_type_stats['event_name'] == 'book_close']['no_of_items'].describe()

count    86796.000000
mean         5.813067
std          8.060027
min          1.000000
25%          1.000000
50%          3.000000
75%          7.000000
max        328.000000
Name: no_of_items, dtype: float64

In [71]:
learner_age_item_type_stats[learner_age_item_type_stats['event_name'] == 'video_close']['no_of_items'].describe()

count    48163.000000
mean         7.624006
std         12.808795
min          1.000000
25%          1.000000
50%          3.000000
75%          8.000000
max        201.000000
Name: no_of_items, dtype: float64

In [72]:
learner_age_item_type_stats[learner_age_item_type_stats['event_name'] == 'audio_close']['no_of_items'].describe()

count    830.000000
mean      13.178313
std       16.983054
min        1.000000
25%        3.000000
50%        8.000000
75%       15.750000
max      119.000000
Name: no_of_items, dtype: float64

In [31]:
dataframe[(dataframe['learner_id'] == 12.0) & (dataframe['event_name'] == 'book_close')]['book_code'].unique()

array(['BO-20140726013738070', 'BO-20140726014932178',
       'BO-20161026170734904', 'BO-20140726015056775',
       'BO-20150417014720158', 'BO-20140726020007169',
       'BO-20150417044756103', 'BO-20150417014720005',
       'BO-20140726020155449', 'BO-20140726020132287'], dtype=object)

In [64]:
dataframe[(dataframe['learner_id'] == 660902.0) & (dataframe['book_code'] == 'BO-20150420190236662')]

,Unnamed: 0,row_id,trial_time,event_name,event_time,receipt_time,learner_id,session_id,book_code,contents_code,library_source,media_fm
15008542,15008543,000000000007EE2A00000000000A15A60000015E84434A...,1970-01-01,video_close,2017-09-15 06:39:16,2017-09-15 06:48:04,660902.0,002738516120170915153105,BO-20150420190236662,CT-20150420190236662,NaN,NaN
15792104,15792105,000000000007EE2A00000000000A15A60000015E894CE2...,1970-01-01,video_close,2017-09-16 06:07:51,2017-09-16 06:18:07,660902.0,002738516120170916150604,BO-20150420190236662,CT-20150420190236662,NaN,NaN
15814338,15814339,000000000007EE2A00000000000A15A60000015E895825...,1970-01-01,video_close,2017-09-16 06:20:09,2017-09-16 06:28:27,660902.0,002738516120170916151946,BO-20150420190236662,CT-20150420190236662,NaN,NaN
14978866,14978867,000000000007EE2A00000000000A15A60000015E93F4DA...,1970-01-01,video_close,2017-09-18 07:47:31,2017-09-18 07:48:15,660902.0,002738516120170918163242,BO-20150420190236662,CT-20150420190236662,NaN,NaN
13730200,13730201,000000000007EE2A00000000000A15A60000015E951365...,1970-01-01,video_close,2017-09-18 13:00:30,2017-09-18 13:12:53,660902.0,002738516120170918212833,BO-20150420190236662,CT-20150420190236662,NaN,NaN
14571484,14571485,000000000007EE2A00000000000A15A60000015E99B31C...,1970-01-01,video_close,2017-09-19 10:33:26,2017-09-19 10:41:26,660902.0,002738516120170919193116,BO-20150420190236662,CT-20150420190236662,NaN,NaN
14957531,14957532,000000000007EE2A00000000000A15A60000015E99BF75...,1970-01-01,video_close,2017-09-19 10:46:55,2017-09-19 10:55:37,660902.0,002738516120170919193116,BO-20150420190236662,CT-20150420190236662,NaN,NaN
12962634,12962635,000000000007EE2A00000000000A15A60000015E9F5E8F...,1970-01-01,video_close,2017-09-20 12:58:48,2017-09-20 13:05:46,660902.0,002738516120170920215432,BO-20150420190236662,CT-20150420190236662,NaN,NaN
13127826,13127827,000000000007EE2A00000000000A15A60000015E9F5F52...,1970-01-01,video_close,2017-09-20 12:59:38,2017-09-20 13:05:58,660902.0,002738516120170920215432,BO-20150420190236662,CT-20150420190236662,NaN,NaN
13264273,13264274,000000000007EE2A00000000000A15A60000015EA4398E...,1970-01-01,video_close,2017-09-21 11:36:29,2017-09-21 11:45:29,660902.0,002738516120170921202841,BO-20150420190236662,CT-20150420190236662,NaN,NaN


In [56]:
learner_books_df = dataframe.groupby(['learner_id', 'book_code'])\
                               .size().reset_index()\
                               .rename(columns={0: 'events_count'})

In [57]:
first_closure_event_df = dataframe.groupby(['learner_id', 'book_code'])['event_time'].min().reset_index().rename(columns={'event_time': 'first_access_time'})

In [58]:
learner_books_df.head()

,learner_id,book_code,events_count
0,5.0,BO-20140726013700796,2
1,5.0,BO-20140726014014719,1
2,12.0,BO-20140726013738070,1
3,12.0,BO-20140726014932178,1
4,12.0,BO-20140726015056775,1


In [52]:
first_closure_event_df[first_closure_event_df['learner_id'] == 268186.0]

,learner_id,book_code,first_access_time
1045558,268186.0,BO-20140726013728499,2017-09-19 07:35:06
1045559,268186.0,BO-20140726014638460,2017-09-14 12:09:25
1045560,268186.0,BO-20140726014719241,2017-09-22 12:03:24
1045561,268186.0,BO-20140726014735858,2017-09-22 11:55:37
1045562,268186.0,BO-20140726015530182,2017-09-19 11:09:00
1045563,268186.0,BO-20140726020215086,2017-09-22 12:24:40
1045564,268186.0,BO-20150417030605438,2017-09-22 06:15:01
1045565,268186.0,BO-20150417044756777,2017-09-19 11:17:32
1045566,268186.0,BO-20150417044757100,2017-09-14 12:09:45
1045567,268186.0,BO-20150728165838280,2017-09-19 11:20:26


In [59]:
learner_books_first_closure_df = learner_books_df.merge(first_closure_event_df, on=['learner_id', 'book_code'])
learner_books_first_closure_df.head()

,learner_id,book_code,events_count,first_access_time
0,5.0,BO-20140726013700796,2,2017-09-19 15:37:10
1,5.0,BO-20140726014014719,1,2017-09-16 08:37:19
2,12.0,BO-20140726013738070,1,2017-09-24 08:22:14
3,12.0,BO-20140726014932178,1,2017-09-24 08:25:50
4,12.0,BO-20140726015056775,1,2017-09-26 09:27:27


In [60]:
learner_books_first_closure_df[learner_books_first_closure_df['learner_id'] == 268186.0]

,learner_id,book_code,events_count,first_access_time
1045558,268186.0,BO-20140726013728499,22,2017-09-19 07:35:06
1045559,268186.0,BO-20140726014638460,4,2017-09-14 12:09:25
1045560,268186.0,BO-20140726014719241,1,2017-09-22 12:03:24
1045561,268186.0,BO-20140726014735858,1,2017-09-22 11:55:37
1045562,268186.0,BO-20140726015530182,1,2017-09-19 11:09:00
1045563,268186.0,BO-20140726020215086,1,2017-09-22 12:24:40
1045564,268186.0,BO-20150417030605438,1,2017-09-22 06:15:01
1045565,268186.0,BO-20150417044756777,1,2017-09-19 11:17:32
1045566,268186.0,BO-20150417044757100,10,2017-09-14 12:09:45
1045567,268186.0,BO-20150728165838280,1,2017-09-19 11:20:26


In [54]:
learner_books_first_closure_df[learner_books_first_closure_df['learner_id'] == 268186.0]

,learner_id,book_code,events_count,first_access_time
1045558,268186.0,BO-20140726013728499,22,2017-09-19 07:35:06
1045559,268186.0,BO-20140726014638460,4,2017-09-14 12:09:25
1045560,268186.0,BO-20140726014719241,1,2017-09-22 12:03:24
1045561,268186.0,BO-20140726014735858,1,2017-09-22 11:55:37
1045562,268186.0,BO-20140726015530182,1,2017-09-19 11:09:00
1045563,268186.0,BO-20140726020215086,1,2017-09-22 12:24:40
1045564,268186.0,BO-20150417030605438,1,2017-09-22 06:15:01
1045565,268186.0,BO-20150417044756777,1,2017-09-19 11:17:32
1045566,268186.0,BO-20150417044757100,10,2017-09-14 12:09:45
1045567,268186.0,BO-20150728165838280,1,2017-09-19 11:20:26


In [40]:
learner_books_first_closure_df[learner_books_first_closure_df['events_count'] > 5]

,learner_id,book_code,events_count,first_access_time
11,12.0,BO-20160122145947170,32,2017-09-24 08:28:08
55,57.0,BO-20161026171155659,11,2017-09-15 16:20:47
58,57.0,BO-20161026171156321,6,2017-09-15 16:30:26
60,57.0,BO-20161026171156481,9,2017-09-14 12:19:01
115,88.0,BO-20150420190234968,15,2017-09-19 08:17:10
120,88.0,BO-20170724102316479,14,2017-09-19 08:17:58
163,155.0,BO-20141105120156248,6,2017-09-20 11:31:48
243,268.0,BO-20150420190234650,6,2017-09-23 10:52:45
262,268.0,BO-20150420190236152,6,2017-09-24 13:34:39
281,268.0,BO-20170328160019995,7,2017-09-14 00:00:37


In [42]:
#666701.0	BO-20170627173817926
#dataframe[(dataframe['learner_id'] == 666701.0) & (dataframe['book_code'] == 'BO-20170627173817926')]['event_time']#.min()
#57.0	BO-20161026171156481
dataframe[(dataframe['learner_id'] == 57.0) & (dataframe['book_code'] == 'BO-20161026171156481')]

,Unnamed: 0,row_id,trial_time,event_name,event_time,receipt_time,learner_id,session_id,book_code,contents_code,library_source,media_fm
15801365,15801366,00000000000206DB00000000000000390000015E8053FB...,1970-01-01,video_close,2017-09-14 12:19:01,2017-09-14 12:29:06,57.0,002011663220170914211152,BO-20161026171156481,NaN,NaN,NaN
15649224,15649225,00000000000206DB00000000000000390000015E80589E...,1970-01-01,video_close,2017-09-14 12:24:05,2017-09-14 12:34:00,57.0,002011663220170914212336,BO-20161026171156481,NaN,NaN,NaN
15263390,15263391,00000000000206DB00000000000000390000015E866962...,1970-01-01,video_close,2017-09-15 16:40:07,2017-09-15 16:52:41,57.0,002011663220170916002016,BO-20161026171156481,NaN,NaN,NaN
13229857,13229858,00000000000206DB00000000000000390000015E9FBAC4...,1970-01-01,video_close,2017-09-20 14:39:31,2017-09-20 14:45:06,57.0,002011663220170920230646,BO-20161026171156481,NaN,NaN,NaN
13149803,13149804,00000000000206DB00000000000000390000015EA047B6...,1970-01-01,video_close,2017-09-20 17:13:28,2017-09-20 17:19:50,57.0,002011663220170920230646,BO-20161026171156481,NaN,NaN,NaN
13278807,13278808,00000000000206DB00000000000000390000015EA4BC85...,1970-01-01,video_close,2017-09-21 13:59:32,2017-09-21 14:05:12,57.0,002011663220170921224721,BO-20161026171156481,NaN,NaN,NaN
16879169,16879170,00000000000206DB00000000000000390000015EB44475...,1970-01-01,video_close,2017-09-24 14:22:19,2017-09-24 14:30:12,57.0,002011663220170924231639,BO-20161026171156481,NaN,NaN,NaN
16879182,16879183,00000000000206DB00000000000000390000015EB4459E...,1970-01-01,video_close,2017-09-24 14:23:35,2017-09-24 14:30:15,57.0,002011663220170924231639,BO-20161026171156481,NaN,NaN,NaN
17647971,17647972,00000000000206DB00000000000000390000015EB445EC...,1970-01-01,video_close,2017-09-24 14:23:55,2017-09-24 14:30:16,57.0,002011663220170924231639,BO-20161026171156481,NaN,NaN,NaN


In [29]:
dataframe[(dataframe['learner_id'] == 12.0) & (dataframe['book_code'] == 'BO-20160122145947170')]#.sort_values(by='event_time')

,Unnamed: 0,row_id,trial_time,event_name,event_time,receipt_time,learner_id,session_id,book_code,contents_code,library_source,media_fm
17397510,17397511,000000000001F005000000000000000C0000015EB30031...,1970-01-01,video_close,2017-09-24 08:28:08,2017-09-24 08:35:22,12.0,002009714920170924172355,BO-20160122145947170,NaN,NaN,NaN
17397597,17397598,000000000001F005000000000000000C0000015EB30166...,1970-01-01,video_close,2017-09-24 08:29:27,2017-09-24 08:35:34,12.0,002009714920170924172355,BO-20160122145947170,NaN,NaN,NaN
16718388,16718389,000000000001F005000000000000000C0000015EB3024C...,1970-01-01,video_close,2017-09-24 08:30:26,2017-09-24 08:40:03,12.0,002009714920170924172355,BO-20160122145947170,NaN,NaN,NaN
16751392,16751393,000000000001F005000000000000000C0000015EB302B6...,1970-01-01,video_close,2017-09-24 08:30:53,2017-09-24 08:40:09,12.0,002009714920170924172355,BO-20160122145947170,NaN,NaN,NaN
17397733,17397734,000000000001F005000000000000000C0000015EB30304...,1970-01-01,video_close,2017-09-24 08:31:13,2017-09-24 08:40:09,12.0,002009714920170924172355,BO-20160122145947170,NaN,NaN,NaN
17123611,17123612,000000000001F005000000000000000C0000015EB3052F...,1970-01-01,video_close,2017-09-24 08:33:35,2017-09-24 08:40:32,12.0,002009714920170924172355,BO-20160122145947170,NaN,NaN,NaN
17397968,17397969,000000000001F005000000000000000C0000015EB305E6...,1970-01-01,video_close,2017-09-24 08:34:22,2017-09-24 08:40:40,12.0,002009714920170924172355,BO-20160122145947170,NaN,NaN,NaN
16718710,16718711,000000000001F005000000000000000C0000015EB3073A...,1970-01-01,video_close,2017-09-24 08:35:49,2017-09-24 08:45:07,12.0,002009714920170924172355,BO-20160122145947170,NaN,NaN,NaN
16718711,16718712,000000000001F005000000000000000C0000015EB3073E...,1970-01-01,video_close,2017-09-24 08:35:50,2017-09-24 08:45:07,12.0,002009714920170924172355,BO-20160122145947170,NaN,NaN,NaN
16751736,16751737,000000000001F005000000000000000C0000015EB3087E...,1970-01-01,video_close,2017-09-24 08:37:12,2017-09-24 08:45:18,12.0,002009714920170924172355,BO-20160122145947170,NaN,NaN,NaN


In [19]:
temp = dataframe.groupby(['learner_id', 'book_code'])['event_time'].min().reset_index()
temp.head()

,learner_id,book_code,event_time
0,5.0,BO-20140726013700796,2017-09-19 15:37:10
1,5.0,BO-20140726014014719,2017-09-16 08:37:19
2,12.0,BO-20140726013738070,2017-09-24 08:22:14
3,12.0,BO-20140726014932178,2017-09-24 08:25:50
4,12.0,BO-20140726015056775,2017-09-26 09:27:27


In [21]:
temp[(temp['learner_id'] == 666701.0) & (temp['book_code'] == 'BO-20170627173817926')]
#12.0	BO-20160122145947170
temp[(temp['learner_id'] == 12.0) & (temp['book_code'] == 'BO-20160122145947170')]

,learner_id,book_code,event_time
11,12.0,BO-20160122145947170,2017-09-24 08:28:08


In [25]:
temp2 = temp.groupby(['learner_id', 'book_code']).size().reset_index()
temp2.head()

,learner_id,book_code,0
0,5.0,BO-20140726013700796,1
1,5.0,BO-20140726014014719,1
2,12.0,BO-20140726013738070,1
3,12.0,BO-20140726014932178,1
4,12.0,BO-20140726015056775,1


In [26]:
temp2[(temp2['learner_id'] == 12.0) & (temp2['book_code'] == 'BO-20160122145947170')]

,learner_id,book_code,0
11,12.0,BO-20160122145947170,1


In [10]:
dataframe.groupby(['learner_id', 'book_code']).size().reset_index()

,learner_id,book_code,0
0,5.0,BO-20140726013700796,2
1,5.0,BO-20140726014014719,1
2,12.0,BO-20140726013738070,1
3,12.0,BO-20140726014932178,1
4,12.0,BO-20140726015056775,1
5,12.0,BO-20140726020007169,1
6,12.0,BO-20140726020132287,1
7,12.0,BO-20140726020155449,1
8,12.0,BO-20150417014720005,1
9,12.0,BO-20150417014720158,1


In [6]:
df.columns

Index(['Unnamed: 0', 'row_id', 'trial_time', 'event_name', 'event_time',
       'receipt_time', 'learner_id', 'session_id', 'book_code',
       'contents_code', 'library_source', 'media_fm'],
      dtype='object')

In [54]:
df['book_code'].describe()

count                 13508060
unique                   11557
top       BO-20170627173817922
freq                    512221
Name: book_code, dtype: object

In [55]:
df['learner_id'].describe()

count    2.364462e+07
mean     3.824910e+05
std      1.815107e+05
min      5.000000e+00
25%      2.348240e+05
50%      4.041670e+05
75%      5.137780e+05
max      6.669300e+05
Name: learner_id, dtype: float64

In [8]:
len(df)

23644617

In [10]:
len(df['row_id'].unique())

23644461

In [12]:
df['row_id'].describe()

count                                              23644617
unique                                             23644461
top       000000000004E4230000000000066B940000015E82461A...
freq                                                      4
Name: row_id, dtype: object

In [13]:
res = df['row_id'].value_counts()
print(res)

000000000004E4230000000000066B940000015E82461A90000000000000015E824EF021    4
000000000006FCAA000000000007CAFF0000015E8247DFB0000000000000015E824EF1EA    4
000000000006903A00000000000766490000015E59538F50000000000000015E824EF039    4
000000000005371D0000000000068B1F0000015E82461A90000000000000015E824EF02F    4
000000000006FCAA000000000007CAFF0000015E82462648000000000000015E824EF041    4
000000000006DBB8000000000007B4520000015E824612C0000000000000015E824EF02D    4
00000000000271B6000000000001B7E70000015E8247ACE8000000000000015E824EF1B5    4
000000000001593E00000000000339290000015E82467850000000000000015E824EF0B5    4
000000000006FCAA000000000007CAFF0000015E8247D3F8000000000000015E824EF1DE    4
000000000006903A00000000000766490000015E64B3C6A8000000000000015E824EF193    4
00000000000765B100000000000735E40000015E8246ED80000000000000015E824EF112    4
000000000001593E00000000000339290000015E82467850000000000000015E824EF0BB    4
000000000002AEFE000000000000C5130000015E8246D228000000000000015E

In [14]:
df[df['row_id'] == '000000000004E4230000000000066B940000015E82461A90000000000000015E824EF021']

,Unnamed: 0,row_id,trial_time,event_name,event_time,receipt_time,learner_id,session_id,book_code,contents_code,library_source,media_fm
18968667,18968668,000000000004E4230000000000066B940000015E82461A...,1970-01-01,video_first_open,2017-09-14 21:23:06,2017-09-14 21:32:44,420756.0,002686229820170915061936,BO-20170821132107586,CT-20170821132107586,TL,MP4
18968668,18968669,000000000004E4230000000000066B940000015E82461A...,1970-01-01,video_first_open,2017-09-14 21:23:06,2017-09-14 21:32:44,420756.0,002686229820170915061936,BO-20170821132107586,CT-20170821132107586,TL,MP4
18968709,18968710,000000000004E4230000000000066B940000015E82461A...,1970-01-01,video_first_open,2017-09-14 21:23:06,2017-09-14 21:32:44,420756.0,002686229820170915061936,BO-20170821132107586,CT-20170821132107586,TL,MP4
18968710,18968711,000000000004E4230000000000066B940000015E82461A...,1970-01-01,video_first_open,2017-09-14 21:23:06,2017-09-14 21:32:44,420756.0,002686229820170915061936,BO-20170821132107586,CT-20170821132107586,TL,MP4


In [16]:
df[df['row_id'] == '000000000006903A00000000000766490000015E64B3C6A8000000000000015E824EF197']

,Unnamed: 0,row_id,trial_time,event_name,event_time,receipt_time,learner_id,session_id,book_code,contents_code,library_source,media_fm
10708216,10708217,000000000006903A00000000000766490000015E64B3C6...,1970-01-01,book_open,2017-09-09 03:34:17,2017-09-14 21:32:45,484937.0,002708098220170915062432,BO-20140726014638460,CT-20140726014638460,TBE,PDFA
10708217,10708218,000000000006903A00000000000766490000015E64B3C6...,1970-01-01,book_open,2017-09-09 03:34:17,2017-09-14 21:32:45,484937.0,002708098220170915062432,BO-20140726014638460,CT-20140726014638460,TBE,PDFA
10708241,10708242,000000000006903A00000000000766490000015E64B3C6...,1970-01-01,book_open,2017-09-09 03:34:17,2017-09-14 21:32:45,484937.0,002708098220170915062432,BO-20140726014638460,CT-20140726014638460,TBE,PDFA
10708242,10708243,000000000006903A00000000000766490000015E64B3C6...,1970-01-01,book_open,2017-09-09 03:34:17,2017-09-14 21:32:45,484937.0,002708098220170915062432,BO-20140726014638460,CT-20140726014638460,TBE,PDFA


In [17]:
len(df['trial_time'].unique())

1

In [18]:
len(df['event_name'].unique())

10

In [20]:
df['event_name'].unique()

array(['audio_close', 'audio_open', 'audio_first_open', 'book_close',
       'book_complete', 'book_first_open', 'book_open', 'video_close',
       'video_first_open', 'video_open'], dtype=object)

In [19]:
df['event_name'].value_counts()

video_close         5401557
video_open          4163399
book_open           4041230
book_close          4004299
book_first_open     2246649
video_first_open    1483684
book_complete       1438767
audio_open           421807
audio_close          407879
audio_first_open      35346
Name: event_name, dtype: int64

In [21]:
no_of_learners = len(df['learner_id'].unique())

264822

In [22]:
len(df['session_id'].unique())

2326130

In [25]:
df['session_id'].describe()

count                     23644617
unique                     2326130
top       002714403420170921203227
freq                          2343
Name: session_id, dtype: object

In [29]:
sessions = df['session_id'].value_counts()
sessions.to_csv('sessions.csv')

In [30]:
df[df['session_id'] == '002735577520170926212651']

,Unnamed: 0,row_id,trial_time,event_name,event_time,receipt_time,learner_id,session_id,book_code,contents_code,library_source,media_fm
390236,390237,000000000007CE7F000000000009EBC20000015EBE4328...,1970-01-01,audio_close,2017-09-26 12:57:06,2017-09-26 13:01:48,650178.0,002735577520170926212651,BO-20150417020356430,NaN,NaN,NaN
390346,390347,000000000007CE7F000000000009EBC20000015EBE2A41...,1970-01-01,audio_close,2017-09-26 12:29:54,2017-09-26 12:32:59,650178.0,002735577520170926212651,BO-20150417020356433,NaN,NaN,NaN
390358,390359,000000000007CE7F000000000009EBC20000015EBE2B1C...,1970-01-01,audio_close,2017-09-26 12:30:50,2017-09-26 12:36:36,650178.0,002735577520170926212651,BO-20150417020356522,NaN,NaN,NaN
390466,390467,000000000007CE7F000000000009EBC20000015EBE335D...,1970-01-01,audio_close,2017-09-26 12:39:51,2017-09-26 12:43:05,650178.0,002735577520170926212651,BO-20150417020356516,NaN,NaN,NaN
390552,390553,000000000007CE7F000000000009EBC20000015EBE3814...,1970-01-01,audio_close,2017-09-26 12:45:00,2017-09-26 12:48:03,650178.0,002735577520170926212651,BO-20150417020356388,NaN,NaN,NaN
390660,390661,000000000007CE7F000000000009EBC20000015EBE404E...,1970-01-01,audio_close,2017-09-26 12:53:59,2017-09-26 12:57:30,650178.0,002735577520170926212651,BO-20150417020356400,NaN,NaN,NaN
390747,390748,000000000007CE7F000000000009EBC20000015EBE2EFC...,1970-01-01,audio_close,2017-09-26 12:35:04,2017-09-26 12:38:11,650178.0,002735577520170926212651,BO-20150417020356424,NaN,NaN,NaN
390857,390858,000000000007CE7F000000000009EBC20000015EBE30D1...,1970-01-01,audio_close,2017-09-26 12:37:04,2017-09-26 12:42:09,650178.0,002735577520170926212651,BO-20150417020356519,NaN,NaN,NaN
391025,391026,000000000007CE7F000000000009EBC20000015EBE3EFA...,1970-01-01,audio_close,2017-09-26 12:52:32,2017-09-26 12:57:05,650178.0,002735577520170926212651,BO-20150417020356513,NaN,NaN,NaN
391183,391184,000000000007CE7F000000000009EBC20000015EBE49D7...,1970-01-01,audio_close,2017-09-26 13:04:24,2017-09-26 13:07:31,650178.0,002735577520170926212651,BO-20150417020356522,NaN,NaN,NaN


In [23]:
len(df['book_code'].unique())

11558

In [24]:
len(df['contents_code'].unique())

10121

In [42]:
user_item_df = df[['learner_id', 'book_code']]

In [44]:
user_item_df['learner_id'].value_counts()

417211.0    13224
342381.0    11911
404783.0    11626
388560.0    10352
639207.0     8826
503664.0     7501
339189.0     7431
419867.0     7262
480005.0     7169
430032.0     7081
453255.0     6361
441531.0     6300
310785.0     6226
192528.0     6022
509307.0     5715
492373.0     5545
472788.0     5323
650178.0     5137
472745.0     4863
637376.0     4748
585428.0     4730
398281.0     4572
445863.0     4151
336525.0     4104
640981.0     4077
252353.0     3921
403964.0     3893
502639.0     3712
259386.0     3700
637441.0     3697
            ...  
377744.0        1
377825.0        1
546800.0        1
547295.0        1
156839.0        1
376879.0        1
43093.0         1
204499.0        1
107859.0        1
156800.0        1
304206.0        1
550357.0        1
156725.0        1
549626.0        1
204777.0        1
43414.0         1
548918.0        1
43472.0         1
156649.0        1
156611.0        1
548294.0        1
303906.0        1
376685.0        1
376779.0        1
547622.0  

In [46]:
learner = df[df['learner_id'] == 417211]
no_of_books = len(learner['book_code'].unique())
print(no_of_books)

65


In [47]:
all_books = len(learner['book_code'])
print(all_books)

13224


In [48]:
df1 = df.groupby(['learner_id','book_code']).size().reset_index().rename(columns={0:'count'})

In [2]:
df1.head()

NameError: name 'df1' is not defined

In [51]:
df1[df1['learner_id'] == 417211]

,learner_id,book_code,count
1837661,417211.0,BO-20150417020356763,206
1837662,417211.0,BO-20150417020356766,208
1837663,417211.0,BO-20150417020356769,212
1837664,417211.0,BO-20150417020356772,206
1837665,417211.0,BO-20150417020356775,211
1837666,417211.0,BO-20150417020356778,208
1837667,417211.0,BO-20150417020356781,209
1837668,417211.0,BO-20150417020356784,197
1837669,417211.0,BO-20150417020356787,208
1837670,417211.0,BO-20150417020356790,202
